In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException, ElementNotInteractableException
from bs4 import BeautifulSoup
import time
import random
import pandas as pd
from tqdm import tqdm

# SETTING UP DRIVER & LINKEDIN LOGIN FUNCTION

In [ ]:
def driver():
    user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'
    s = Service(ChromeDriverManager().install())
    options = webdriver.ChromeOptions()
    options.add_argument('--ignore-certificate-errors')
    options.add_argument('--incognito')
    #options.add_argument("headless")
    options.add_argument(f'user-agent={user_agent}')
    options.add_argument("--window-size=1920,1080")
    #options.binary_location = '/Applications/Brave Browser.app/Contents/MacOS/Brave Browser'
    return webdriver.Chrome(service=s, options=options)
    
def linkedin_login():
    mail = 'cocazapata.21@gmail.com'
    pw = 'fcbarcelona21'
    driver.get('https://www.linkedin.com')
    username = driver.find_element(By.ID, 'email-or-phone').send_keys(mail)
    password = driver.find_element(By.ID, 'password')
    password.send_keys(pw)
    password.send_keys(Keys.ENTER)

# SCRAPPING LINKEDIN 

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import random
from bs4 import BeautifulSoup
from tqdm import tqdm
import pandas as pd
import numpy as np

def data_analyst():
    user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'
    s = Service(ChromeDriverManager().install())
    options = webdriver.ChromeOptions()
    options.add_argument('--ignore-certificate-errors')
    options.add_argument('--incognito')
    # options.add_argument("headless")
    options.add_argument(f'user-agent={user_agent}')
    options.add_argument("--window-size=1920,1080")
    options.binary_location = '/Applications/Brave Browser.app/Contents/MacOS/Brave Browser'
    driver = webdriver.Chrome(service=s, options=options)
    mail = 'cocazapata.21@gmail.com'
    pw = 'fcbarcelona21'
    driver.get('https://www.linkedin.com')
    username = driver.find_element(By.ID, 'session_key').send_keys(mail)
    password = driver.find_element(By.ID, 'session_password').send_keys(pw)
    #password.send_keys(Keys.ENTER)

    time.sleep(20)

    list_of_dictionaries = []

    for i in tqdm(range(0, 976, 25)):
        #driver.get(f'''https://www.linkedin.com/jobs/search/?f_TPR=r2592000&geoId=107025191&keywords=data&location=Barcelona%2C%20Catalonia%2C%20Spain&start={i}''')
        driver.get(f'''https://www.linkedin.com/jobs/search/?currentJobId=3652284026&geoId=107025191&keywords=data&location=Barcelona%2C%20Catalonia%2C%20Spain&start={i}''')
        time.sleep(random.randint(3, 5))
        all_listings = []
        keep_scrolling = True

        while keep_scrolling:
            listings = driver.find_elements(By.CSS_SELECTOR, ".job-card-list__title")
            if set(listings) == set(all_listings):
                keep_scrolling = False
            else:
                new_listings = list(set(listings) - set(all_listings))
                for listing in new_listings:
                    all_listings.append(listing)
                    try:
                        driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth'});", listing)
                        time.sleep(random.randint(1, 3))
                        listing.click()
                        time.sleep(random.randint(2, 5))
                        page_source = driver.page_source
                        soup = BeautifulSoup(page_source, "html.parser")
                        try:
                            posicioncita  = {}
                            title = driver.find_element(By.XPATH, "//*[starts-with(@id, 'ember')]/h2").text
                            posicioncita["title"] = title
                            
                            if 'data' in title.lower() or 'datos' in title.lower():
                                
                                try:
                                    company = driver.find_element(By.XPATH, "//*[contains(@class, 'ember-view t-black t-normal')]").text
                                    posicioncita["companies"] = company
                                except:
                                    posicioncita["companies"] = np.nan
                                try:
                                    location = driver.find_element(By.XPATH, "//*[contains(@class, 'jobs-unified-top-card__bullet')]").text
                                    posicioncita["location"] = location
                                except:
                                    posicioncita["location"] = np.nan
                                try:
                                    modality = soup.select('span.jobs-unified-top-card__workplace-type')[0].get_text()
                                    posicioncita["modality"] = modality
                                except:
                                    posicioncita["modality"] = np.nan
                                try:
                                    description = soup.select('#job-details > span')[0].get_text()
                                    posicioncita["description"] = description
                                except:
                                    posicioncita["description"] = np.nan
                            
                            list_of_dictionaries.append(posicioncita)
                                    
                        except NoSuchElementException:
                            driver.save_screenshot(f"{random.randint(0, 100)}.png")
                    except StaleElementReferenceException:
                        driver.save_screenshot(f"{random.randint(0, 100)}.png")
                time.sleep(random.randint(1, 2))
    driver.quit()
    df = pd.DataFrame(list_of_dictionaries)
    return df

In [ ]:
analyst = data_analyst()

In [ ]:
analyst

# QUICK CLEANING & CONCATING DIFFERENT DF FROM DIFFERENT SCRAPPING TIMES

In [ ]:
analyst.drop_duplicates(subset=['description'], inplace=True)


In [ ]:
analyst.info()

In [ ]:
analyst.rename(columns={'companies': 'company'}, inplace=True)


In [ ]:
# Deleting all rows that contains more than 2 NaN values:
nan_counts = combined_df.isna().sum(axis=1)

filtered_df = analyst[nan_counts < 2]
print(filtered_df)

In [ ]:
df2 = pd.read_csv('DATA_JOBS.csv')

In [ ]:
df1 = pd.read_csv('DATA_JOBS_BCN.csv')

In [ ]:
df_final = pd.concat([df1, df2], ignore_index=True)


In [ ]:
df_final.drop_duplicates(subset=['description'], inplace=True)


In [ ]:
df_final.info()

In [ ]:
df_final.to_csv("DATA_JOBS_JULIO.csv", index=False)
